<a href="https://colab.research.google.com/github/RO-AD/waymo-od-motion-pred/blob/main/tutorial/1_transformer/gp-transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformer 실습
참고 : [Attention is All You Need Tutorial(ndb796)](https://github.com/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/Attention_is_All_You_Need_Tutorial_(German_English).ipynb)

자연어처리 끝판왕. 다른 분야에도 고르게 사용됨.

In [2]:
# 아,,, colab에서 UTF-8 오류 뜨는데 아래 코드로 해결함,,, 찝찝,,,,하네
import locale
locale.getpreferredencoding = lambda: "UTF-8"

## 자연어를 어떻게 Input으로 줄 수 있을까?
기본적으로 모델의 INPUT은 숫자 형태로 들어가기 때문에 처리하고자 하는 문장을 분해 및 숫자로 변환하는 전처리 작업이 필요하다. 이 과정을 문장의 토큰화(tokenization)이라고 한다.

**torchtext**는 자연어처리에 유용한 라이브러리로, 데이터셋과 전처리 클래스, 스코어링 함수 등이 존재한다. 설치해보자!

In [3]:
%%capture
!pip install torchtext==0.6.0

이번에는 **spaCy** 라이브러리를 이용해보자. **토큰화(tokenization), 태깅(tagging) 등의 전처리**를 위한 라이브러리다. 영어와 한국어 전처리 모듈을 설치하고 확인해보자

In [4]:
%%capture
!python -m spacy download en_core_web_sm
!python -m spacy download ko_core_news_sm

In [5]:
import spacy

spacy_en = spacy.load('en_core_web_sm') # 영어 토큰화(tokenization)
spacy_ko = spacy.load('ko_core_news_sm') # 한국어 토큰화(tokenization)

In [6]:
# 간단히 토큰화(tokenization) 기능 써보기
_tmp_en_tokens = spacy_en.tokenizer("I graduated last week.")
_tmp_ko_tokens = spacy_ko.tokenizer("저는 저번 주에 졸업했습니다.")

# 영어 문장 토큰화
for i, token in enumerate(_tmp_en_tokens):
    print(f"인덱스 {i}: {token.text}")

print("--------------")

# 한국어 문장 토큰화
for i, token in enumerate(_tmp_ko_tokens):
    print(f"인덱스 {i}: {token.text}")

인덱스 0: I
인덱스 1: graduated
인덱스 2: last
인덱스 3: week
인덱스 4: .
--------------
인덱스 0: 저는
인덱스 1: 저번
인덱스 2: 주에
인덱스 3: 졸업했습니다
인덱스 4: .


토큰화 과정을 함수화 해보자

In [7]:
tokenizer_en = lambda text: [token.text for token in spacy_en.tokenizer(text)]
tokenizer_ko = lambda text: [token.text for token in spacy_ko.tokenizer(text)]

In [8]:
tokenizer_en('I graduated last week.')

['I', 'graduated', 'last', 'week', '.']

필드(field)를 이용해 데이터셋에 대한 구체적인 전처리 과정을 명시할 수 있다. 이 작업을 통해 추후 코드에서 전처리를 조금 더 단순하게 다룰 수 있다.

In [56]:
from torchtext.data import Field

# 목표 : ko -> en 로 번역
src_field = Field(tokenize=tokenizer_ko, init_token="<sos>", eos_token="<eos>", lower=True, batch_first=True)
trg_field = Field(tokenize=tokenizer_en, init_token="<sos>", eos_token="<eos>", lower=True, batch_first=True)

**Parameter**
- tokenize : 어떤 토큰화 함수를 적용할 것인지 지정(default : split)
- init_token : ?
- eos_token : ?
- lower : 대문자가 있다면 소문자로 변경할 것인지
- batch_first : ?(미니배치 차원을 맨 앞으로 하여 데이터를 불러올 것인지)

이제 데이터셋을 불러오자. **torchtext의 datasets**에는 대표적인 번역 데이터셋인 **Mullti30k**이 있다. 였는데,,, 아쉽게도 한국어는 없어서 다른 라이브러리를 사용함.

여러 데이터셋을 모아서 라이브러리화 하는 **Korpora**라는 라이브러리가 있었음

In [18]:
%%capture
!pip install Korpora

In [19]:
from Korpora import Korpora
Korpora.fetch("korean_parallel_koen_news")
datasets = Korpora.load("korean_parallel_koen_news")

[Korpora] Corpus `korean_parallel` is already installed at /root/Korpora/korean_parallel/korean-english-park.train.tar.gz
decompress /root/Korpora/korean_parallel/korean-english-park.train.tar.gz
[Korpora] Corpus `korean_parallel` is already installed at /root/Korpora/korean_parallel/korean-english-park.dev.tar.gz
decompress /root/Korpora/korean_parallel/korean-english-park.dev.tar.gz
[Korpora] Corpus `korean_parallel` is already installed at /root/Korpora/korean_parallel/korean-english-park.test.tar.gz
decompress /root/Korpora/korean_parallel/korean-english-park.test.tar.gz

    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : KakaoBrain
    Repository : https://github.com/jungyeul/korean-parallel-corpora
    References :
        - Jungyeul Park, Jeen-Pyo Hong and Jeon

In [20]:
datasets.train

koennews.train: size=94123
  - koennews.train.texts : list[str]
  - koennews.train.pairs : list[str]

In [21]:
datasets.test

koennews.test: size=2000
  - koennews.test.texts : list[str]
  - koennews.test.pairs : list[str]

불러온 데이터셋을 예제를 따라가기 편한 구조로 변경해주었다. 기존의 `torchtext.datasets.translation.Multi30k` 구조가 `torchtext.data.Example`의 집합을 `torchtext.data.Dataset`으로 묶어준 형태여서 이것을 그대로 재현했다.

In [60]:
from torchtext.data import Example

test_dataset_examples = []

for text, pair in zip(datasets.test.texts, datasets.test.pairs):
  example = Example.fromlist([text, pair], [("src", src_field), ("trg", trg_field)])
  test_dataset_examples.append(example)

train_dataset_examples = []

for text, pair in zip(datasets.test.texts, datasets.test.pairs):
  example = Example.fromlist([text, pair], [("src", src_field), ("trg", trg_field)])
  train_dataset_examples.append(example)

In [61]:
from torchtext.data import Dataset

train_dataset = Dataset(examples=train_dataset_examples, fields=[("src", src_field), ("trg", trg_field)])
test_dataset  = Dataset(examples=test_dataset_examples,  fields=[("src", src_field), ("trg", trg_field)])

In [62]:
# 학습 데이터 중 하나를 선택해 출력
print(vars(train_dataset.examples[30])['src'])
print(vars(train_dataset.examples[30])['trg'])

['이', '박테리아들은', '로켓이나', '위성에', '실려', '우주로', '갔을', '수도', '있으며', ',', '아니면', '정말로', '다른', '행성에서', '온', '것일', '수도', '있다', '.']
['the', 'bacteria', 'could', 'have', 'hitched', 'a', 'ride', 'on', 'a', 'rocket', 'or', 'satellite', 'into', 'space', 'or', 'they', 'really', 'could', 'be', 'from', 'another', 'planet', '.']


이렇게 토큰화된 dataset을 구비했다. 이제 각 단어를 숫자 형태로 변경해주어야 한다. **Field**의 **build_vocab** 메서드를 이용해 사전을 만들 수 있고, 이를 기반으로 숫자로 변경한다.

In [63]:
src_field.build_vocab(train_dataset, min_freq=2)
trg_field.build_vocab(train_dataset, min_freq=2)

print(f"src vocab len : {len(src_field.vocab)}")
print(f"trg vocab len : {len(trg_field.vocab)}")

src vocab len : 4162
trg vocab len : 4020


**Parameter**
- min_freq : 최소 2번 이상 등장한 단어

In [84]:
trg_field.vocab.itos[4]

'the'

In [65]:
print(trg_field.vocab.stoi["hihihihi"]) # 없는 단어: 0
print(trg_field.vocab.stoi[trg_field.pad_token]) # 패딩(padding): 1
print(trg_field.vocab.stoi["<sos>"]) # : 2
print(trg_field.vocab.stoi["<eos>"]) # : 3
print(trg_field.vocab.stoi["a"])
print(trg_field.vocab.stoi["world"])

0
1
2
3
8
93


vocab 내에서 각 데이터를 숫자로 변환할 수 있는 것을 확인할 수 있었다. 그러나 각 문장 내 단어의 개수가 각기 다르기 때문에 BucketIterator를 이용하여 패딩을 포함한 동일한 개수로 맞춰줄 수 있다.
- 배치 크기(batch size): 128

In [74]:
from torchtext.data import BucketIterator

BATCH_SIZE = 128

# 일반적인 데이터 로더(data loader)의 iterator와 유사하게 사용 가능
train_iterator, test_iterator = BucketIterator.splits(
    (train_dataset, test_dataset),
    batch_size=BATCH_SIZE,
    device='cuda')

In [79]:
for i, batch in enumerate(train_iterator):
    src = batch.src
    trg = batch.trg

    print(f"첫 번째 배치 크기: {src.shape}")

    # 현재 배치에 있는 하나의 문장에 포함된 정보 출력
    for i in range(src.shape[1]):
        print(f"인덱스 {i}: {src[0][i].item()}") # 여기에서는 [Seq_num, Seq_len]

    # 첫 번째 배치만 확인
    break

첫 번째 배치 크기: torch.Size([128, 42])
인덱스 0: 2
인덱스 1: 92
인덱스 2: 1381
인덱스 3: 334
인덱스 4: 0
인덱스 5: 0
인덱스 6: 598
인덱스 7: 0
인덱스 8: 659
인덱스 9: 3463
인덱스 10: 904
인덱스 11: 125
인덱스 12: 15
인덱스 13: 4
인덱스 14: 3
인덱스 15: 1
인덱스 16: 1
인덱스 17: 1
인덱스 18: 1
인덱스 19: 1
인덱스 20: 1
인덱스 21: 1
인덱스 22: 1
인덱스 23: 1
인덱스 24: 1
인덱스 25: 1
인덱스 26: 1
인덱스 27: 1
인덱스 28: 1
인덱스 29: 1
인덱스 30: 1
인덱스 31: 1
인덱스 32: 1
인덱스 33: 1
인덱스 34: 1
인덱스 35: 1
인덱스 36: 1
인덱스 37: 1
인덱스 38: 1
인덱스 39: 1
인덱스 40: 1
인덱스 41: 1
